# Time-optimal path planning
This notebook handles standard code for the computation of time-optimal paths in windfields

In [1]:
import os
import sys
import time
import numpy as np
from math import atan, cos, sin
sys.path.extend(['/home/bastien/Documents/work/mermoz', 
                 '/home/bastien/Documents/work/mermoz/src',
                 '/home/bastien/Documents/work/mdisplay',
                 '/home/bastien/Documents/work/mdisplay/src'])

from mermoz.feedback import TargetFB
from mermoz.mdf_manager import MDFmanager
from mermoz.params_summary import ParamsSummary
from mermoz.problem import MermozProblem
from mermoz.model import ZermeloGeneralModel
from mermoz.solver import Solver
from mermoz.stoppingcond import TimedSC, DistanceSC
from mermoz.wind import DiscreteWind
from mermoz.misc import *
from mdisplay.geodata import GeoData

from solver_utils import select_wind

%load_ext autoreload
%autoreload

### Problem definition

Initial point, target, UAV airspeed

In [2]:
# Scale factor
sf = 1.

# Initial point
x_init = sf * np.array((0.6, 0.6))

# Target
x_target = sf * np.array((2.4, 1.4))

# Windfield boundaries
bl = sf * np.array((0.5, 0.5))
tr = sf * np.array((2.5, 2.5))

# UAV airspeed in m/s
v_a = 0.05

case_name = 'double-gyre-kularatne2016'

Set up the problem

In [3]:
# Prepare output directory
output_dir = f'/home/bastien/Documents/work/mermoz/output/example_solver_{case_name}'
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

# Create a file manager to dump problem data
mdfm = MDFmanager()
mdfm.set_output_dir(output_dir)

# Problem coordinates
# coords = COORD_GCS
coords = COORD_CARTESIAN

# Problem wind data
from mermoz.wind import DoubleGyreWind
total_wind = DoubleGyreWind(0.5, 0.5, 2., 2., pi * 0.02)
def domain(x):
    return bl[0] < x[0] < tr[0] and bl[1] < x[1] < tr[1]

# Time window upper bound
# Estimated through great circle distance + 20 percent
T = 40.

# Creates the cinematic model
zermelo_model = ZermeloGeneralModel(v_a, coords=coords)
zermelo_model.update_wind(total_wind)

# Creates the navigation problem on top of the previous model
mp = MermozProblem(zermelo_model, T=T, coords=coords, autodomain=False, domain=domain)

### Solver parameters

In [4]:
u_min = DEG_TO_RAD * -179.
u_max = DEG_TO_RAD * 180.

nt_pmp = 1000
opti_ceil = 0.05
N_dirs = 50
neighb_ceil = 0.9*opti_ceil

solver = Solver(mp,
                x_init,
                x_target,
                T,
                u_min,
                u_max,
                output_dir,
                N_disc_init=N_dirs,
                opti_ceil=opti_ceil,
                neighb_ceil=neighb_ceil,
                n_min_opti=1,
                adaptive_int_step=False,
                N_iter=nt_pmp)

solver.log_config()
solver.setup()

Solve problem

In [5]:
t_start = time.time()
solver.solve_fancy()
t_end = time.time()
time_pmp = t_end - t_start

Shooting -179.0
Shooting -171.7
Shooting -164.3
Shooting -157.0
Shooting -149.7
Shooting -142.4
Shooting -135.0
Shooting -127.7
Shooting -120.4
Shooting -113.1
Shooting -105.7
Shooting -98.4
Shooting -91.1
Shooting -83.8
Shooting -76.4
Shooting -69.1
Shooting -61.8
Shooting -54.4
Shooting -47.1
Shooting -39.8
Shooting -32.5
Shooting -25.1
Shooting -17.8
Shooting -10.5
Shooting -3.2
Shooting 4.2
Shooting 11.5
Shooting 18.8
Shooting 26.1
Shooting 33.5
Shooting 40.8
Shooting 48.1
Shooting 55.4
Shooting 62.8
Shooting 70.1
Shooting 77.4
Shooting 84.8
Shooting 92.1
Shooting 99.4
Shooting 106.7
Shooting 114.1
Shooting 121.4
Shooting 128.7
Shooting 136.0
Shooting 143.4
Shooting 150.7
Shooting 158.0
Shooting 165.3
Shooting 172.7
Shooting 180.0
0.91631, 81.09
0.99754, 88.42
1.54267, 37.13
1.23307, 73.77
1.80237, -28.81
1.67080, 0.50
1.57214, 29.81
1.46853, 51.79
1.34692, 66.44
1.95440, -50.79
1.86500, -36.13
1.96977, -160.68
1.69511, -6.83
1.64727, 7.83
1.59875, 22.48
1.96869, -58.11
1.50890, 44

### Explicit control laws

In [46]:
sc = DistanceSC(lambda x: np.linalg.norm(x - x_target), opti_ceil)
sc = TimedSC(T)
from mermoz.feedback import FixedHeadingFB, ConstantFB

mp.load_feedback(FixedHeadingFB(mp.model.wind, v_a, 0., mp.coords))
mp.integrate_trajectory(x_init, sc, int_step=T / (nt_pmp - 1))

mp.load_feedback(ConstantFB(0.))
mp.integrate_trajectory(x_init, sc, int_step=T / (nt_pmp - 1))

Dump everything to output directory

In [6]:
nx = 51
ny = 51
mdfm.dump_wind(total_wind, nx=nx, ny=ny, bl=bl, tr=tr)
mdfm.dump_trajs(mp.trajs)
ps = ParamsSummary({}, output_dir)
ps.load_from_solver(solver)
ps.add_param('bl_wind', tuple(bl))
ps.add_param('tr_wind', tuple(tr))
#ps.add_param('nx_wind', nx)
#ps.add_param('ny_wind', ny)
ps.add_param('nt_pmp', nt_pmp)
ps.add_param('pmp_time', time_pmp)
ps.dump()